In [ ]:
# specify all top-level input parameters

# central values
a0 = 0.01            # 0.85e-9 * lambda [microns] * Sqrt(Intensity [W/cm^2])
lambda0 = 8.e-7      # wavelength [m]
tau_fwhm = 0.3e-13   # FWHM pulse length [s]
w0 = 400.e-6         # waist size [m]
z_waist = 0.0        # longitudinal waist position [m]
z_center = 0.0       # longitudinal position of pulse center [m]

# deviations
d_lambda = 0.01     # full wavelength chirp across laser pulse [m]
dw0x = 0.0           # deviation of horizontal waist from w0
dw0y = 0.0           # deviation of vertical waist from w0
dzwx = 0.0           # deviation of horizontal waist position from z_waist
dzwy = 0.0           # deviation of vertical waist position from z_waist
x_shift = 0.         # bulk horizontal shift of pulse position
y_shift = 0.         # bulk vertical shift of pulse position

# numerical values
num_slices = 3       # desired number of slices (i.e. SRW wavefronts) to represent the pulse
n_points = 101       # number of bins along each axis for contour plots

In [ ]:
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as const

from pykern.pkcollections import PKDict
import rslaser.rsoptics.wavefront as rswf
import rslaser.rscavity.laser_cavity as rslc

import srwlib
from srwlib import srwl

In [ ]:
# create a dictionary of all relevant physical and numerical quantities
data_in=PKDict()

data_in.a0 = a0
data_in.lambda0 = lambda0
data_in.tau_fwhm = tau_fwhm
data_in.w0 = w0
data_in.z_waist = z_waist
data_in.z_center = z_center

data_in.d_lambda = d_lambda
data_in.dw0x = dw0x
data_in.dw0y = dw0y
data_in.dzwx = dzwx
data_in.dzwy = dzwy
data_in.x_shift = x_shift
data_in.y_shift = y_shift

data_in.nslice = num_slices

#create laser pulse
LP = rslc.LaserPulse(data_in)

In [ ]:
# choose one of the laser pulse slices, and grab its SRW wavefront object
slice=LP.slice
wfr0=slice[0].wfr

In [ ]:
#Plot initial wavefront
##Initial wfr calculation
arIinit = srwlib.array('f', [0]*wfr0.mesh.nx*wfr0.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIinit, wfr0, 6, 0, 3, wfr0.mesh.eStart, 0, 0) #extracts intensity

##Reshaping electric field data from flat to 2D array
arIinit2D = np.array(arIinit).reshape((wfr0.mesh.nx, wfr0.mesh.ny), order='C')
wfrsizei=np.size(arIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvalsf02=np.linspace(wfr0.mesh.xStart,wfr0.mesh.xFin,wfr0.mesh.nx)
yvalsf02=np.linspace(wfr0.mesh.yStart,wfr0.mesh.yFin,wfr0.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

# By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
plt.pcolormesh(np.multiply(xvalsf02,1e6), np.multiply(yvalsf02,1e6), arIinit2D, cmap=plt.cm.viridis,shading='auto')
plt.colorbar()
#['Horizontal Position [mm]', , 'Intensity']
ax.set_ylim(-1e4,1e4)
ax.set_xlim(-1e4,1e4)
ax.set_ylabel(r'Vertical Position [$\mu m$]')
ax.set_xlabel(r'Horizontal Position [$\mu m$]')
ax.set_title('Intensity at Waist')
plt.show()

In [ ]:
# create a Gaussian wavefront via SRW
# is w0 the same as sigrW ...?
phE = const.h * const.c / data_in.lambda0
propLen = 0.
pulseE = 1.
poltype = 0
wfr0=rswf.createGsnSrcSRW(data_in.w0,propLen,pulseE,poltype,phE)